<a href="https://colab.research.google.com/github/siddharth-0704/Sentiment_analysis/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install --upgrade torch torchvision --extra-index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.7.1%2Bcu118-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (28 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.22.1%2Bcu118-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (6.1 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached https://download.pytorch.org/whl/triton-3.3.1-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (1.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.3/905.3 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 50.2 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstal

In [1]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [2]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 109.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.1
    Uninstalling transformers-4.53.1:
      Successfully uninstalled transformers-4.53.1


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [5]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')
result = model(tokens)
result.logits
int(torch.argmax(result.logits))+1

4

In [11]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.trustpilot.com/review/www.nike.com'
headers = {
    'User-Agent': 'Mozilla/5.0'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Trustpilot review text is inside <p> with data-service-review-text-* class
reviews = []
for tag in soup.find_all('p'):
    if tag.get('data-service-review-text-translation-variant') or len(tag.text.strip()) > 50:
        reviews.append(tag.text.strip())

# Print top 5 reviews
for i, review in enumerate(reviews[:5], 1):
    print(f"{i}. {review}\n")


1. Companies on Trustpilot can’t offer incentives or pay to hide any reviews.

2. Most reviewers were let down by their experience overall. Consumers express significant dissatisfaction with various aspects of the company's services. A recurring issue revolves around order fulfillment, where people report problems with their purchases. Customer service is another sore point, with many reviewers describing their interactions as unhelpful and frustrating.

Additionally, a significant number of consumers have had negative experiences with refunds, citing delays and complications in the process. The delivery service also receives criticism, as people have reported issues such as late deliveries or parcels being delivered to the wrong address. Contacting the company to resolve these issues appears to be difficult, further compounding the frustration of consumers.See more

3. Bought Nike football boots at  sports direct they were worn twice and the sole cracked. Returned to Nike as it is a f

In [12]:
import numpy as np
import pandas as pd
df = pd.DataFrame(np.array(reviews), columns=['review'])




In [13]:
df['review'].iloc[0]

'Companies on Trustpilot can’t offer incentives or pay to hide any reviews.'

In [14]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [15]:
sentiment_score(df['review'].iloc[1])

2

In [16]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [17]:
df

,review,sentiment
0,Companies on Trustpilot can’t offer incentives...,1
1,Most reviewers were let down by their experien...,2
2,Bought Nike football boots at sports direct t...,1
3,Ordered some slides dpd asked for safe place i...,1
4,Non-existent customer service. Returned cloth...,1
5,"I last wrote a year ago. My order at the time,...",1
6,"Experience sports, training, shopping and ever...",5
7,"Niels Brocks Gade 8 , 4. sal., 8900, Randers C...",5
8,Bought Nike football boots at sports direct t...,1
9,"I last wrote a year ago. My order at the time,...",1
